# 🪙 Cryptocurrency Market Analysis Template (with Retry)

This notebook pulls price data from **CoinGecko** and builds a concise market analysis.

### Features
- Price history (up to 365 days)
- Daily returns, cumulative returns
- Rolling 30D volatility
- Max drawdown per asset
- Simple Sharpe-like metric
- Correlations
- Matplotlib charts

### Fix for 429 Too Many Requests
We added `safe_fetch_price_history` with retry & delay handling.


In [ ]:
# %pip install requests pandas numpy matplotlib --quiet
import requests, time, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 5)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 160)

VS_CURRENCY = "usd"
DAYS = 365
COINS = ["bitcoin", "ethereum", "solana", "binancecoin", "ripple"]

def fetch_markets(coin_ids):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": VS_CURRENCY,
        "ids": ",".join(coin_ids),
        "order": "market_cap_desc",
        "per_page": len(coin_ids),
        "page": 1,
        "sparkline": False,
        "price_change_percentage": "24h,7d,30d"
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return pd.DataFrame(r.json())

def fetch_price_history(coin_id, days=DAYS):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart"
    params = {"vs_currency": VS_CURRENCY, "days": days}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    prices = data.get("prices", [])
    if not prices:
        return pd.DataFrame(columns=["date", coin_id])
    df = pd.DataFrame(prices, columns=["ts", coin_id])
    df['date'] = pd.to_datetime(df['ts'], unit='ms').dt.date
    df = df.drop(columns=['ts']).set_index('date')
    return df

def safe_fetch_price_history(coin_id, days=DAYS, retries=3, delay=2):
    for attempt in range(retries):
        try:
            return fetch_price_history(coin_id, days)
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                print(f"Rate limit hit for {coin_id}, retrying in {delay}s...")
                time.sleep(delay)
            else:
                raise
    print(f"Failed to fetch {coin_id} after {retries} retries.")
    return pd.DataFrame()

def compute_drawdown(series):
    cummax = series.cummax()
    return series / cummax - 1.0

def sharpe_like(returns, periods_per_year=365):
    mu = returns.mean() * periods_per_year
    sigma = returns.std() * np.sqrt(periods_per_year)
    return np.where(sigma == 0, np.nan, mu / sigma)

# Market snapshot
markets = fetch_markets(COINS)
display_cols = ["id", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h", "price_change_percentage_7d_in_currency", "price_change_percentage_30d_in_currency"]
snapshot = markets.reindex(columns=display_cols)
snapshot = snapshot.rename(columns={
    "id": "asset",
    "symbol": "ticker",
    "current_price": f"price_{VS_CURRENCY}",
    "market_cap": f"market_cap_{VS_CURRENCY}",
    "total_volume": f"volume_{VS_CURRENCY}",
    "price_change_percentage_24h": "chg_24h_%",
    "price_change_percentage_7d_in_currency": "chg_7d_%",
    "price_change_percentage_30d_in_currency": "chg_30d_%"
})
snapshot

In [ ]:
price_tables = []
for coin in COINS:
    df = safe_fetch_price_history(coin, DAYS)
    price_tables.append(df)
    time.sleep(1.5)
prices = pd.concat(price_tables, axis=1)
prices = prices.fillna(method='ffill').dropna(how='all')
prices.tail()

In [ ]:
returns = prices.pct_change().dropna()
cum_returns = (1 + returns).cumprod() - 1
rolling_vol_30d = returns.rolling(window=30).std() * np.sqrt(365)
drawdowns = prices.apply(compute_drawdown)
max_drawdown = drawdowns.min()
risk_table = pd.DataFrame({
    "sharpe_like": sharpe_like(returns),
    "vol_30d_ann": rolling_vol_30d.tail(1).T.squeeze(),
    "max_drawdown": max_drawdown
}).sort_values(by="sharpe_like", ascending=False)
risk_table

In [ ]:
corr = returns.corr()
corr

In [ ]:
prices.plot(title=f'Price History ({DAYS}d)'); plt.show()

In [ ]:
rolling_vol_30d.plot(title='Rolling 30D Volatility (Annualized)'); plt.show()

In [ ]:
drawdowns.plot(title='Drawdowns'); plt.show()

In [ ]:
outdir = "crypto_market_outputs"
os.makedirs(outdir, exist_ok=True)
snapshot.to_csv(os.path.join(outdir, "snapshot.csv"), index=False)
prices.to_csv(os.path.join(outdir, "prices.csv"))
returns.to_csv(os.path.join(outdir, "returns.csv"))
cum_returns.to_csv(os.path.join(outdir, "cum_returns.csv"))
rolling_vol_30d.to_csv(os.path.join(outdir, "rolling_vol_30d.csv"))
drawdowns.to_csv(os.path.join(outdir, "drawdowns.csv"))
corr.to_csv(os.path.join(outdir, "correlations.csv"))
risk_table.to_csv(os.path.join(outdir, "risk_table.csv"))
print("Saved CSVs to:", os.path.abspath(outdir))

## Next steps
- Add more coins to `COINS`
- Use longer retry/delay if still hitting rate limits
- Combine with portfolio analytics
